In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np

VPRM_old_or_new = "old"
version = "V4"

region = "Europe" #  "Alps" or  "Europe"
# run_ID = "Europe_VPRM_"+VPRM_old_or_new+"_optimized_params_diff_evo_V2_100"
# base_path = "/home/madse/Downloads/Fluxnet_Data/all_tuned_params/" + run_ID +"/"+region


run_ID = region+"_VPRM_optimized_params_diff_evo_"+version+"_100"
base_path = "/home/madse/Downloads/Fluxnet_Data/all_tuned_params/" + run_ID 

folders = [
    f
    for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f))
]
flx_folders = [folder for folder in folders if folder.startswith("FLX_")]

if not flx_folders:
    print("Warning - There is no input data")
    raise SystemExit(0)

df_parameters = pd.DataFrame()


# Loop through each FLX_ folder and append data from XLSX files
for folder in flx_folders:
    folder_path = os.path.join(base_path, folder)
    files = [f for f in os.listdir(folder_path) if f.endswith(VPRM_old_or_new+'_diff_evo_'+version+'_100.xlsx')]
    for file in files:
        file_path = os.path.join(folder_path, file)
        data = pd.read_excel(file_path)
        df_parameters = pd.concat([df_parameters, data], axis=0)


In [ ]:

# List of tuples containing site_ID and Year values to drop
values_to_drop = [
    ("NL-Hor", 2006),
    ("NL-Hor", 2011),
    ("OT-Ro2", 2009),
    ("IT-Ro1", 2000),
    ("IT-Ro1", 2007),
    ("IT-Noe", 2004),
    ("IT-Noe", 2011),
    ("IT-Col", 2003),
    ("IT-Col", 2004),
    ("IT-Col", 2006),
    ("IT-CA3", 2011),
    ("IT-CA2", 2011),
    ("IT-CA1", 2011),
    ("FR-Gri", 2004),
    ("FI-Let", 2009),
    ("FI-Jok", 2003),
    ("ES-LJu", 2004),
    ("ES-LgS", 2009),
    ("ES-Amo", 2007),
    ("ES-Amo", 2008),
    ("DK-Sor", 2014),
    ("DK-Eng", 2005),
    ("DE-Zrk", 2013),
    ("DE-Spw", 2010),
    ("DE-Spw", 2013),
    ("DE-Seh", 2007),
    ("DE-Seh", 2010),
    ("DE-Spw", 2010),
    ("DE-Spw", 2013),
    ("DE-Seh", 2007),
    ("DE-Seh", 2010),
    ("DE-RuS", 2011),
    ("DE-RuS", 2012),
    ("DE-RuR", 2011),
    ("DE-Kli", 2004),
    ("DE-Kli", 2009),
    ("DE-Akm", 2009),
    ("DE-Akm", 2011),
    ("DE-Akm", 2012),
    ("DE-Akm", 2014),
    ("CZ-BK2", 2006),
    ("CZ-BK2", 2007),
    ("CZ-BK2", 2008),
    ("DE-Akm", 2009),
#Alps
    ("BE-Bra", 2003),
    ("CH-Cha", 2009),
    ("CH-Cha", 2005),
    ("CH-Dav", 2005),
    ("CH-Fru", 2005),
    ("CH-Fru", 2006),
    ("CH-Lae", 2004),
    ("CZ-wet", 2006),
    ("DE-Lkb", 2009),
    ("DE-Lkb", 2010),
    ("DE-Lkb", 2013),
    ("DE-SfN", 2012),
    ("IT-La2", 2000),
    ("IT-La2", 2002),
    ("IT-PT1", 2002),
    ("IT-Ren", 2004),
    ("IT-Ren", 2001),
    ("IT-Ren", 2000),
    ("IT-Tor", 2008)
]

df_parameters['site_year'] = df_parameters['site_ID'] + '_' + df_parameters['Year'].astype(str)
for site_year in set(df_parameters['site_year']).intersection(set('_'.join(map(str, tpl)) for tpl in values_to_drop)):
    df_parameters = df_parameters[df_parameters['site_year'] != site_year]
df_parameters.drop(columns=['site_year'], inplace=True)

# # Create a new column by concatenating site_ID and PFT
# df_parameters['new_site_ID'] = df_parameters['site_ID'] + '_' + df_parameters['PFT']
# df_parameters['site_ID'] = df_parameters['new_site_ID']
# df_parameters.drop(columns=['new_site_ID'], inplace=True)


In [ ]:
sns.color_palette('bright')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

df_parameters_nn = df_parameters.copy()
df_parameters_nn = df_parameters_nn.dropna()

color_palette = sns.color_palette('bright')
pft_colors = {
    'CRO': color_palette[8], 
    'DBF': color_palette[0], 
    'ENF': color_palette[9],
    'MF': color_palette[4], 
    'GRA': color_palette[2],     
    'SHB': color_palette[7],
    'WET': color_palette[3]
}

# df_parameters_nn["diff_unc"] = df_parameters_nn["RANDUNC"]/df_parameters_nn["JOINTUNC"]
df_parameters_nn["diff_unc"] = df_parameters_nn["JOINTUNC"]-df_parameters_nn["RANDUNC"]

plt.figure(figsize=(8, 6))
plt.scatter(df_parameters_nn["diff_unc"], df_parameters_nn["R2_NEE"], alpha=0.5, c=df_parameters_nn["PFT"].map(pft_colors))
coefficients = np.polyfit(df_parameters_nn["diff_unc"], df_parameters_nn["R2_NEE"], 1)
poly = np.poly1d(coefficients)
# plt.plot(df_parameters_nn["diff_unc"], poly(df_parameters_nn["diff_unc"]), color='red', label=f'y = {coefficients[0]:.2f}x + {coefficients[1]:.2f}')
equation_regression = f'y = {coefficients[0]:.2f}x + {coefficients[1]:.2f}'
equation_normal = 'y = x'
plt.xlabel("JOINTUNC-RANDUNC")
plt.ylabel("R2_NEE")
plt.grid(True)

for pft, color in pft_colors.items():
    plt.scatter([], [], c=color, label=pft)  # Create an empty scatter plot for each PFT label

plt.legend()

plt.tight_layout()
plt.savefig(
    base_path 
    + "/regression_JRUNC_vs_R2_NEE_"
    + VPRM_old_or_new
    + "_"
    + run_ID
    + ".eps",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:

print(f"Number of deleted site years due to R2_NEE < 0 = {sum(df_parameters['R2_NEE'] < 0)}")

# TODO: NSE is identical find research on comparison to R2: sum(df_parameters["NSE_NEE"] < 0)
df_parameters = df_parameters[df_parameters["R2_NEE"] > 0]
df_parameters.reset_index(drop=True, inplace=True)

In [ ]:

plt.figure(figsize=(8, 6))
plt.scatter(df_parameters["T_mean"], df_parameters["Topt"], alpha=0.5, c=df_parameters["PFT"].map(pft_colors))
coefficients = np.polyfit(df_parameters["T_mean"], df_parameters["Topt"], 1)
poly = np.poly1d(coefficients)
plt.plot(df_parameters["T_mean"], poly(df_parameters["T_mean"]), color='red', label=f'y = {coefficients[0]:.2f}x + {coefficients[1]:.2f}')
equation_regression = f'y = {coefficients[0]:.2f}x + {coefficients[1]:.2f}'
equation_normal = 'y = x'
plt.xlabel("T_mean")
plt.ylabel("Topt")
plt.grid(True)

for pft, color in pft_colors.items():
    plt.scatter([], [], c=color, label=pft)  # Create an empty scatter plot for each PFT label

plt.legend()

plt.tight_layout()
plt.savefig(
    base_path 
    + "/regression_Topt_vs_Tmean_"
    + VPRM_old_or_new
    + "_"
    + run_ID
    + ".eps",
    dpi=300,
    bbox_inches="tight",
)


In [ ]:


plt.figure(figsize=(8, 6))
plt.scatter(df_parameters_nn["T_max"].dropna(), df_parameters_nn["Topt"].dropna(), alpha=0.5, c=df_parameters_nn["PFT"].map(pft_colors))
coefficients = np.polyfit(df_parameters_nn["T_max"], df_parameters_nn["Topt"], 1)
poly = np.poly1d(coefficients)
plt.plot(df_parameters_nn["T_max"], poly(df_parameters_nn["T_max"]), color='red', label=f'y = {coefficients[0]:.2f}x + {coefficients[1]:.2f}')
equation_regression = f'y = {coefficients[0]:.2f}x + {coefficients[1]:.2f}'
equation_normal = 'y = x'
plt.xlabel("T_max")
plt.ylabel("Topt")
plt.grid(True)

for pft, color in pft_colors.items():
    plt.scatter([], [], c=color, label=pft)  # Create an empty scatter plot for each PFT label

plt.legend()

plt.tight_layout()
plt.savefig(
    base_path 
    + "/regression_Topt_vs_Tmax_"
    + VPRM_old_or_new
    + "_"
    + run_ID
    + ".eps",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
if VPRM_old_or_new == "new":
    parameters_to_plot = [
        "Topt",
        "PAR0",
        "lambd",
        "alpha1",
        "alpha2",
        "beta",
        "T_crit",
        "T_mult",
        "gamma",
        "theta1",
        "theta2",
        "theta3",
    ]
else:
    parameters_to_plot = [
        "Topt",
        "PAR0",
        "lambd",
        "alpha",
        "beta",
    ]


In [ ]:
# Define the color palette and the PFT color mapping

df_parameters.sort_values(by="PFT", inplace=True)
color_palette = sns.color_palette('bright')
# Create a list of colors for the boxplot based on the sorted PFTs
pft_order = df_parameters['PFT'].unique()
colors = [pft_colors[pft] for pft in pft_order]


In [ ]:

if VPRM_old_or_new == "new":
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 20))
else:
    fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(10, 30))

axes = axes.flatten()
for i, parameter in enumerate(parameters_to_plot):
    sns.boxplot(x='PFT', y=parameter, data=df_parameters, ax=axes[i],palette=colors) 
    sns.swarmplot(x='PFT', y=parameter, data=df_parameters, color="0.25", alpha=0.5, ax=axes[i])
    axes[i].set_title(f'{parameter} by PFT')
    axes[i].set_xlabel('PFT')
    axes[i].set_ylabel(parameter)
    axes[i].tick_params(axis='x', rotation=45)

handles = []
for pft, color in pft_colors.items():
    handles.append(plt.scatter([], [], c=color, label=pft))

plt.legend(handles=handles, bbox_to_anchor=(0.5, -0.2), loc='upper center', ncol=len(pft_colors)//2, fontsize='small')

plt.tight_layout()
plt.savefig(
    base_path 
    + "/boxplot_PFTs_"
    + VPRM_old_or_new
    + "_"
    + run_ID
    + ".eps",
    dpi=300,
    bbox_inches="tight",
)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Create a dictionary mapping site_ID to PFT
site_to_pft = df_parameters.set_index('site_ID')['PFT'].to_dict()

# Create a list of colors for each site based on the PFT
site_colors = [pft_colors[site_to_pft[site]] for site in df_parameters['site_ID'].unique()]

if VPRM_old_or_new == "new":
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 20))
else:
    fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(10, 30))

axes = axes.flatten()

for i, parameter in enumerate(parameters_to_plot):
    sns.boxplot(x='site_ID', y=parameter, data=df_parameters, ax=axes[i], palette=site_colors)
    sns.swarmplot(x='site_ID', y=parameter, data=df_parameters, color="0.15", alpha=0.5, ax=axes[i])
    axes[i].set_title(f'{parameter} by site_ID')
    axes[i].set_ylabel(parameter)
    axes[i].tick_params(axis='x', rotation=45)

# Create legend handles
handles = []
for pft, color in pft_colors.items():
    handles.append(plt.scatter([], [], c=color, label=pft))

plt.legend(handles=handles, bbox_to_anchor=(0.5, -0.2), loc='upper center', ncol=len(pft_colors)//2, fontsize='small')

plt.tight_layout()
plt.savefig(
    base_path +
    "/boxplot_siteIDs_"
    + VPRM_old_or_new
    + "_"
    + run_ID
    + ".eps",
    dpi=300,
    bbox_inches="tight",
)
plt.show()


In [ ]:
grouped = df_parameters.groupby('PFT')
dfs_to_concat = []
for parameter in parameters_to_plot:
    for pft, group_data in grouped:
        mean = group_data[parameter].mean()
        median = group_data[parameter].median()
        # Create a DataFrame with the new row
        new_row = pd.DataFrame({'PFT': [pft], 'Parameter': [parameter], 'Mean': [mean], 'Median': [median]})
        # Append the new DataFrame to the list
        dfs_to_concat.append(new_row)

mean_median_df = pd.concat(dfs_to_concat, ignore_index=True)
mean_median_df.to_excel(base_path + "/mean_median_params_"+ VPRM_old_or_new+ "_"+run_ID+".xlsx", index=False)

In [ ]:

# Pivoting the DataFrame
pivoted_mean = mean_median_df.pivot(index='Parameter', columns='PFT', values='Mean')
pivoted_median = mean_median_df.pivot(index='Parameter', columns='PFT', values='Median')

# Exporting to CSV
pivoted_mean.to_csv(base_path + "/"+region+"_parameters_mean_"+ VPRM_old_or_new+ "_"+run_ID+".csv")
pivoted_median.to_csv(base_path + "/"+region+"_parameters_median_"+ VPRM_old_or_new+ "_"+run_ID+".csv")

In [ ]:

parameters_to_plot = ['R2_GPP', 'R2_Reco', 'R2_NEE', 'RMSE_GPP', 'RMSE_Reco', 'RMSE_NEE']

grouped = df_parameters.groupby('PFT')
dfs_to_concat = []
for parameter in parameters_to_plot:
    for pft, group_data in grouped:
        mean = group_data[parameter].mean()
        median = group_data[parameter].median()
        # Create a DataFrame with the new row
        new_row = pd.DataFrame({'PFT': [pft], 'Parameter': [parameter], 'Mean': [mean], 'Median': [median]})
        # Append the new DataFrame to the list
        dfs_to_concat.append(new_row)

mean_median_df = pd.concat(dfs_to_concat, ignore_index=True)
mean_median_df.to_excel(base_path + "/mean_median_R2_RMSE_"+ VPRM_old_or_new+ "_"+run_ID+".xlsx", index=False)

parameters_to_plot = [ 'R2_NEE', 'RMSE_NEE']

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
axes = axes.flatten()

for i, parameter in enumerate(parameters_to_plot):
    sns.boxplot(x='PFT', y=parameter, data=df_parameters, ax=axes[i],palette=colors)
    sns.swarmplot(x='PFT', y=parameter, data=df_parameters, color="0.25", alpha=0.5, ax=axes[i])
    axes[i].set_title(f'{parameter} by PFT')
    #axes[i].set_xlabel('PFT')
    axes[i].set_ylabel(parameter)
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(
    base_path +
    "/boxplot_R2_RMSE_"
    + VPRM_old_or_new
    + "_"
    + run_ID
    + ".eps",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
# try to plot colors according to PFT
# import seaborn as sns
# import matplotlib.pyplot as plt

# color_palette = 'tab20'  # 'muted', 'deep', 'husl'
# sns.set_palette(color_palette)

# if VPRM_old_or_new == "new":
#     fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 20))
# else:
#     fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(10, 30))

# axes = axes.flatten()

# # Create a list to store legend handles and labels
# legend_handles = []

# for i, parameter in enumerate(parameters_to_plot):
#     sns.boxplot(x='PFT', y=parameter, data=df_parameters, ax=axes[i])
#     sns.swarmplot(x='PFT', y=parameter, data=df_parameters, color="0.25", alpha=0.5, ax=axes[i], dodge=True)
#     axes[i].set_title(f'{parameter} by PFT')
#     axes[i].set_xlabel('PFT')
#     axes[i].set_ylabel(parameter)
#     axes[i].tick_params(axis='x', rotation=45)
    
#     # Add handles and labels for legend
#     handles, labels = axes[i].get_legend_handles_labels()
#     legend_handles.extend(handles)

# plt.tight_layout(rect=[0, 0.05, 1, 1])  # Adjust layout to accommodate legend below all the tiles

# plt.savefig(
#     base_path 
#     + "/boxplot_PFTs_"
#     + VPRM_old_or_new
#     + "_"
#     + run_ID
#     + ".eps",
#     dpi=300,
#     bbox_inches="tight",
# )

# if VPRM_old_or_new == "new":
#     fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 20))
# else:
#     fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(10, 30))

# axes = axes.flatten()

# for i, parameter in enumerate(parameters_to_plot):
#     sns.boxplot(x='site_ID', y=parameter, data=df_parameters, hue='PFT', palette=color_palette, ax=axes[i])
#     sns.swarmplot(x='site_ID', y=parameter, data=df_parameters, hue='PFT', palette=color_palette, alpha=0, dodge=True, ax=axes[i])
#     axes[i].set_title(f'{parameter} by site_ID')
#     axes[i].set_xlabel('site_ID')
#     axes[i].set_ylabel(parameter)
#     axes[i].tick_params(axis='x', rotation=45)
# fig.legend(legend_handles, labels, loc='lower center', ncol=len(set(df_parameters['PFT'])), bbox_to_anchor=(0.5, -0.05))
# plt.tight_layout()

# plt.savefig(
#     base_path +
#     "/boxplot_siteIDs_"
#     + VPRM_old_or_new
#     + "_"
#     + run_ID
#     + ".eps",
#     dpi=300,
#     bbox_inches="tight",
# )
# plt.show()


In [ ]:
# if VPRM_old_or_new == "new":
#     fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 20))
# else:
#     fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(10, 30))  

# axes = axes.flatten()

# mean_median_df = pd.DataFrame(columns=['Parameter', 'Mean', 'Median'])

# for i, parameter in enumerate(parameters_to_plot):
#     q_low = df_parameters[parameter].quantile(0.05)
#     q_high = df_parameters[parameter].quantile(0.95)
#     filtered_data = df_parameters[(df_parameters[parameter] >= q_low) & (df_parameters[parameter] <= q_high)]
#     sns.boxplot(x='PFT', y=parameter, data=filtered_data, ax=axes[i])
#     sns.swarmplot(x='PFT', y=parameter, data=filtered_data, color="0.25", alpha=0.5, ax=axes[i])
#     axes[i].set_title(f'{parameter} by PFT')
#     axes[i].set_xlabel('PFT')
#     axes[i].set_ylabel(parameter)
#     axes[i].tick_params(axis='x', rotation=45)
    
# plt.tight_layout()
# plt.savefig(base_path + "/boxplot_PFTs_filtered_with_median_"+run_ID+".eps", dpi=300, bbox_inches="tight")
# plt.show()
